In [1]:
import torch
import shutil
import pickle

from simulation import citygraph_dataset
# from learning import inductive_route_learning, eval_route_generator, bee_colony
from learning.bee_colony import main_bee # я так обозвал
from learning.eval_route_generator import main_eval # я так обозвал
from omegaconf import OmegaConf, DictConfig
from hydra import initialize_config_dir, compose
from simulation import drawing

from tqdm import tqdm
from pathlib import Path

ImportError: cannot import name 'main_bee' from 'learning.bee_colony' (/root/TNDP_learning/learning/bee_colony.py)

In [ ]:
from hydra import initialize_config_dir, compose
from omegaconf import OmegaConf
import os

cfg_dir = os.path.abspath("../TNDP_learning/cfg")


### Обучение LC

In [ ]:
with initialize_config_dir(config_dir=cfg_dir, job_name="app"):
    cfg_learn = compose(config_name="ppo_20nodes_copy.yaml", # в этот файл можно положить путь к пикл файлу с графами, также можно попробовать другие параметры на обучение
                            overrides=["+run_name=train_1"]) # при желании можно накинуть ему имя 


/var/folders/zx/byb3bcds50s1lwbdr2mwq18r0000gn/T/ipykernel_33090/4288414268.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(config_dir=cfg_dir, job_name="app"):


In [5]:
cfg_learn

{'ppo': {'n_iterations': 200, 'val_period': 10, 'n_epochs': 1, 'minibatch_size': 128, 'horizon': 120, 'epsilon': 0.2, 'use_gae': True, 'gae_lambda': 0.95}, 'discount_rate': 0.95, 'diff_reward': True, 'baseline_lr': 0.0005, 'entropy_weight': 0.0, 'batch_size': 128, 'reward_scale': 1.0, 'lr': 0.0016134816080499328, 'decay': 0.0008404361781997002, 'optimizer': 'Adam', 'eval': {'n_routes': 10, 'min_route_len': 8, 'max_route_len': 30}, 'dataset': {'type': 'pickle', 'kwargs': {'path': 'output_graphs/mixed', 'space_scale': 0.6, 'demand_scale': 0.2}}, 'experiment': {'logdir': 'training_logs', 'anomaly': False, 'cpu': False, 'seed': 0, 'symmetric_routes': True, 'cost_function': {'type': 'mine', 'kwargs': {'mean_stop_time_s': 0, 'avg_transfer_wait_time_s': 300, 'demand_time_weight': 0.5, 'route_time_weight': 0.5, 'constraint_violation_weight': 5.0, 'variable_weights': True, 'pp_fraction': 0.33, 'op_fraction': 0.33}}}, 'model': {'common': {'dropout': 0.0, 'nonlin_type': 'ReLU', 'embed_dim': 64}, 

In [ ]:
inductive_route_learning.setup_and_train(cfg_learn) # веса для модели вернутся в папку output, там будет файл .pt / при желании можно накинуть ему имя 

### Стартовый набор маршрутов

In [ ]:
with initialize_config_dir(config_dir=cfg_dir, version_base=None):
    
    cfg_eval = compose(
        config_name="eval_model_mumford",   # из @hydra.main
        overrides=[
            "+eval=mandl", # конфиг в котором задается кол-во маршрутов и их макс и мин длины (можно заменить на vo для теста по ваське)
            "+model.weights=../TNDP_learning/output/inductive_30_06_2025_18_12_45.pt", # путь к весам модели
            "++eval.dataset.path=../TNDP_learning/datasets/mumford_dataset/Instances", # путь к данным (можно заменить на ваську)
            "++eval.dataset.city=Mandl", # название города из датасетов, при тесте на ваське написать VO
            "++run_name=test" # имя запуска, вернет pickle файл с тензором в output_routes
        ]
    )

In [ ]:
print(OmegaConf.to_yaml(cfg)) # чисто проверка как выглядит конфиг 

In [ ]:
main_eval(cfg)

### Генерация 

In [4]:
with initialize_config_dir(config_dir=cfg_dir, version_base=None):
    cfg_bco = compose(
        config_name="bco_mumford",  # как в @hydra.main для bee_colony.py
        overrides=[
            "+eval=mandl",
            "eval.dataset.path=../TNDP_learning/datasets/mumford_dataset/Instances",
            "init.path=output_routes/nn_construction_mandl_lc100_test_routes.pkl",
        ]
    )

In [5]:
cfg_bco

{'n_bees': 10, 'n_type1_bees': 5, 'n_iterations': 400, 'batch_size': 10, 'force_linking_unlinked': False, 'experiment': {'logdir': 'training_logs', 'anomaly': False, 'cpu': False, 'seed': 0, 'symmetric_routes': True, 'cost_function': {'type': 'mine', 'kwargs': {'mean_stop_time_s': 0, 'avg_transfer_wait_time_s': 300, 'demand_time_weight': 0.5, 'route_time_weight': 0.5, 'constraint_violation_weight': 5.0, 'variable_weights': True, 'pp_fraction': 0.33, 'op_fraction': 0.33}}}, 'init': {'method': 'load', 'path': 'output_routes/nn_construction_mandl_lc100_test_routes.pkl'}, 'eval': {'csv': True, 'n_routes': 6, 'min_route_len': 2, 'max_route_len': 8, 'dataset': {'type': 'mumford', 'path': '../TNDP_learning/datasets/mumford_dataset/Instances', 'city': 'Mandl'}}}

In [ ]:
main_bee(cfg_bco)

100%|██████████| 1/1 [00:33<00:00, 33.34s/it]

,6.000,0.558,13.493,140.000,59.216,36.609,4.110,0.064,0.000,0.000,33.336,401.000


In [13]:
with initialize_config_dir(config_dir=cfg_dir, version_base=None):
    cfg_neural = compose(
        config_name="neural_bco_mumford",
        overrides=[
            "+eval=mandl",
            "++eval.dataset.path=../TNDP_learning/datasets/mumford_dataset/Instances",
            "++model.weights=../TNDP_learning/output/inductive_30_06_2025_18_12_45.pt",
            # Куда сохранять initial routes (init.method по defaults = load)
            "init.path=output_routes/nn_construction_mandl_lc100_test_routes.pkl",
        ]
    )

In [9]:
cfg_neural

{'n_bees': 10, 'n_iterations': 400, 'batch_size': 10, 'neural_bees': True, 'force_linking_unlinked': False, 'experiment': {'logdir': 'training_logs', 'anomaly': False, 'cpu': False, 'seed': 0, 'symmetric_routes': True, 'cost_function': {'type': 'mine', 'kwargs': {'mean_stop_time_s': 0, 'avg_transfer_wait_time_s': 300, 'demand_time_weight': 0.5, 'route_time_weight': 0.5, 'constraint_violation_weight': 5.0, 'variable_weights': True, 'pp_fraction': 0.33, 'op_fraction': 0.33}}}, 'model': {'common': {'dropout': 0.0, 'nonlin_type': 'ReLU', 'embed_dim': 64}, 'route_generator': {'kwargs': {'force_linking_unlinked': False, 'logit_clip': None, 'n_nodepair_layers': 3, 'n_pathscorer_layers': 3, 'pathscorer_hidden_dim': 16, 'n_halt_layers': 3, 'halt_scorer_type': 'endpoints', 'serial_halting': True}, 'type': 'PathCombiningRouteGenerator'}, 'backbone_gn': {'net_type': 'graph attn', 'kwargs': {'n_layers': 5, 'in_node_dim': 4, 'in_edge_dim': 13, 'use_norm': False, 'n_heads': 4, 'dense': False}}, 'weig

In [ ]:
main_bee(cfg_neural)